# NFL Players per State

In this analysis I am going to analyze which states have produced the most and least active NFL players in the current 2023-2024 season. 
I will do this by gathering data regarding the birthplaces of NFL players from Football Reference here: https://www.pro-football-reference.com/friv/birthplaces.htm and then joining
state population data from the US Census Bureau from here: https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html#v2023.

I will then calculate how many NFL players each state produces per million inhabitants to determine which states over- and under-produce NFL talent.



In [96]:
import pandas as pd
import lxml
from pandasql import sqldf
import plotly.express as px

df = pd.read_html('https://www.pro-football-reference.com/friv/birthplaces.htm#birthplaces')
df1 = df[0]
df1 = pd.DataFrame(df1)

df1.head()


,Rk,Country,State,# of Pros,# Active,# of HOF,G,TDs,Most Touchdowns,TDs.1,Most Games Played,Games
0,1,Unknown,NaN,394,356,0,6146,341,NaN,NaN,NaN,NaN
1,2,Unknown,FL,1,1,0,2,0,NaN,NaN,NaN,NaN
2,3,USA,NaN,267,0,0,1467,56,Marty Norton,10.0,Jake Williams,44.0
3,4,USA,AK,15,2,0,976,17,Steve Smith,12.0,Mark Schlereth,156.0
4,5,USA,AL,788,69,10,41251,2132,Terrell Owens,156.0,Philip Rivers,244.0


In [97]:
#Only want rows where player is from USA

df1 = df1.loc[df1['Country'] == 'USA']

df1.head()



,Rk,Country,State,# of Pros,# Active,# of HOF,G,TDs,Most Touchdowns,TDs.1,Most Games Played,Games
2,3,USA,NaN,267,0,0,1467,56,Marty Norton,10.0,Jake Williams,44.0
3,4,USA,AK,15,2,0,976,17,Steve Smith,12.0,Mark Schlereth,156.0
4,5,USA,AL,788,69,10,41251,2132,Terrell Owens,156.0,Philip Rivers,244.0
5,6,USA,American Samoa,41,5,0,2180,34,Mosi Tatupu,21.0,Domata Peko,214.0
6,7,USA,AR,323,8,6,15711,1614,Don Hutson,105.0,Trey Junkin,281.0


In [138]:
#Drop NaN state

df1 = df1.loc[df1['State'].notna()]

df1.head()

,Rk,Country,State,# of Pros,# Active,# of HOF,G,TDs,Most Touchdowns,TDs.1,Most Games Played,Games
3,4,USA,Alaska,15,2,0,976,17,Steve Smith,12.0,Mark Schlereth,156.0
4,5,USA,Alabama,788,69,10,41251,2132,Terrell Owens,156.0,Philip Rivers,244.0
5,6,USA,American Samoa,41,5,0,2180,34,Mosi Tatupu,21.0,Domata Peko,214.0
6,7,USA,Arkansas,323,8,6,15711,1614,Don Hutson,105.0,Trey Junkin,281.0
7,8,USA,Arizona,214,23,2,11851,508,Todd Heap,42.0,J.J. Jansen,243.0


In [99]:
#Adding census data for states: from this link: https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html#v2023

url = 'https://www2.census.gov/programs-surveys/popest/datasets/2020-2023/state/totals/NST-EST2023-ALLDATA.csv'

df2 = pd.read_csv(url)

df2.head()


,SUMLEV,REGION,DIVISION,STATE,NAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,POPESTIMATE2023,...,RNATURALCHG2023,RINTERNATIONALMIG2021,RINTERNATIONALMIG2022,RINTERNATIONALMIG2023,RDOMESTICMIG2021,RDOMESTICMIG2022,RDOMESTICMIG2023,RNETMIG2021,RNETMIG2022,RNETMIG2023
0,10,0,0,0,United States,331464948,331526933,332048977,333271411,334914895,...,1.510043,1.133278,3.003867,3.409196,0.000000,0.000000,0.000000,1.133278,3.003867,3.409196
1,20,1,0,0,Northeast Region,57614141,57430477,57243423,57026847,56983517,...,0.976280,1.403074,3.678035,3.947167,-4.823207,-7.881683,-5.671414,-3.420133,-4.203648,-1.724247
2,30,1,1,0,New England,15119994,15057898,15106108,15120739,15159777,...,-0.019551,1.772643,4.532659,5.024221,1.615767,-3.086263,-2.445863,3.388409,1.446396,2.578358
3,30,1,2,0,Middle Atlantic,42494147,42372579,42137315,41906108,41823740,...,1.336417,1.271165,3.370662,3.557656,-7.121462,-9.606391,-6.837920,-5.850297,-6.235729,-3.280264
4,20,2,0,0,Midwest Region,68987296,68969794,68850246,68783028,68909283,...,0.665803,0.802684,2.098649,2.409866,-2.577042,-2.636615,-1.245226,-1.774357,-0.537966,1.164640


In [100]:
#Take State and 2023 Population

query = '''SELECT [NAME] AS [State], [POPESTIMATE2023] AS [Population]
            FROM df2'''

demo = sqldf(query)

demo.head(15)

,State,Population
0,United States,334914895
1,Northeast Region,56983517
2,New England,15159777
3,Middle Atlantic,41823740
4,Midwest Region,68909283
5,East North Central,47146039
6,West North Central,21763244
7,South Region,130125290
8,South Atlantic,68225883
9,East South Central,19700801


In [101]:
#Drop first 13 columns, as they are not states

demo.drop(index=demo.index[:14], axis=0, inplace=True)

demo.head()

,State,Population
14,Alabama,5108468
15,Alaska,733406
16,Arizona,7431344
17,Arkansas,3067732
18,California,38965193


In [135]:
#Turning abbreviated states to full names in nfl data, this will also drop US territories, which we want as we


    #List of states
def stateFull(df, col):
    abbrev2state = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming',
        'PR': 'Puerto Rico',
        'VI': 'Virigin Islands'
    }

    df1['State'] = df1[col].str.strip().replace(abbrev2state)

stateFull(df1, 'State')

df1.head()

,Rk,Country,State,# of Pros,# Active,# of HOF,G,TDs,Most Touchdowns,TDs.1,Most Games Played,Games
3,4,USA,Alaska,15,2,0,976,17,Steve Smith,12.0,Mark Schlereth,156.0
4,5,USA,Alabama,788,69,10,41251,2132,Terrell Owens,156.0,Philip Rivers,244.0
5,6,USA,American Samoa,41,5,0,2180,34,Mosi Tatupu,21.0,Domata Peko,214.0
6,7,USA,Arkansas,323,8,6,15711,1614,Don Hutson,105.0,Trey Junkin,281.0
7,8,USA,Arizona,214,23,2,11851,508,Todd Heap,42.0,J.J. Jansen,243.0


In [134]:
#Join population to the nfl stats table and calculation active players per capita, this will get rid of the U.S territories, which we want as this is focused on the 50 states
#Taking removing column about historical stats, as we are only focused on active players


query = '''SELECT df1.[State], df1.[# Active], demo.[Population]
           FROM df1
           JOIN demo ON df1.State=demo.State
           '''

nfl = sqldf(query)

nfl.head()

,State,# Active,Population
0,Alaska,2,733406
1,Alabama,69,5108468
2,Arkansas,8,3067732
3,Arizona,23,7431344
4,California,206,38965193


In [124]:
#Drop Puerto Rico
nfl = nfl.loc[nfl['State'] != 'Puerto Rico']

In [141]:
#Calculate active player per million in each state

nfl['Active Players per Million'] = nfl['# Active'] / (nfl['Population']/1000000)

nfl.head()

,State,# Active,Population,Active Players per Million
0,Alaska,2,733406,2.727003
1,Alabama,69,5108468,13.506985
2,Arkansas,8,3067732,2.607790
3,Arizona,23,7431344,3.094999
4,California,206,38965193,5.286770


In [128]:
#Summary statistics

nfl.describe()

,# Active,Population,Active Players per Million
count,51.000000,5.100000e+01,51.000000
mean,38.176471,6.566959e+06,5.342018
std,49.515939,7.460690e+06,3.875277
min,0.000000,5.840570e+05,0.000000
25%,6.500000,1.867398e+06,2.724599
50%,19.000000,4.526154e+06,4.398612
75%,48.500000,7.622112e+06,6.556887
max,206.000000,3.896519e+07,18.147039


In [153]:
#Total number of active players in the nfl

print(nfl['# Active'].sum())

1947


In [149]:
print(nfl['Active Players per Million'].median())

4.374831973672013


### Observations from descriptive statistics

The average active players per state per million is 5.34

Min value shows that there are states that have no active players in the nfl

Can see that there is a large range between the highest and lowest producing states

The total number of active players is 1947

In [137]:
#Top 5 states for overall active players

query2 = '''SELECT [State], [# Active]
            FROM nfl
            GROUP BY [State]
            ORDER BY [# Active] DESC'''

top5 = sqldf(query2)

top5.head()

,State,# Active
0,California,206
1,Texas,192
2,Florida,182
3,Georgia,157
4,Ohio,87


California, Texas and Florida are the top 3 states for most nfl talent. This isn't very suprising due to their large populations and warm climates.

In [144]:
#Plotting the values to see overall distribution

fig = px.bar(nfl, y='# Active', x='State')

fig.show()

From this chart it is prevalent that the Florida, California, Texas and Georgia produce a significant amount of the talent in the NFL, comprising 737 players or 38% of the NFL.

In [146]:
#Top 5 States for active players per million

query3 = '''SELECT [State], [Active Players per Million]
            FROM nfl
            GROUP BY [State]
            ORDER BY [Active Players per Million] DESC '''

top5_per = sqldf(query3)

top5_per.head()

,State,Active Players per Million
0,Louisiana,18.147039
1,Mississippi,17.008596
2,Georgia,14.234905
3,Alabama,13.506985
4,District of Columbia,11.782518


From this we can see that Georgia is still in the top 5 but the other four have dropped out

It is also apparent that the top four of these states are from the "Bible Belt", states famous for their football culture, with universities in the NCAA's Southeastern Conference, the 
biggest conference in NCAA Football

In [121]:
#Plotting the values to see overall distribution

fig = px.bar(nfl, y='Active Players per Million', x='State')

fig.show()

The chart shows that Alabama, District of Columbia, Georgia, Louisiana and Mississippi produce the highest amount of players per million, with Hawauu close behind.

Additionally, it is prevalent that Maine, New Hampshire and Rhode Island produce significantly less players per million that other states and that Vermont has no active players - 

These findings could indicate that difference could be down to regional cultural attitudes towards football (ie. Southern States, vs. New England), with District of Columbia being an exception.



In [130]:
#Export csv to visualize with PowerBI

nfl.to_csv('nfl_states', encoding='UTF-8')